# SQL Project
The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

We've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

**Purposes of the analysis:** to build the next queries that would help a value proposition for a new product:
- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help us exclude brochures and similar publications from the analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.

## Initialization

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import Image

In [2]:
# Establish connection to the Data Base
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Description of the data

Here is the description of the tables from the data base

**books**

Contains data on books:

- `book_id`
- `author_id`
- `title`
- `num_pages` — number of pages
- `publication_date`
- `publisher_id`

**authors**

Contains data on authors:

- `author_id`
- `author`

**publishers**

Contains data on publishers:

- `publisher_id`
- `publisher`

**ratings**

Contains data on user ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

**reviews**

Contains data on customer reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book  
- `text` — the text of the review

### Data Diagram

In [3]:
# Specify the URL of the uploaded image
image_url = 'https://imgur.com/953oZmL.png'

# Display the image
Image(url=image_url)


### Studying the tables
Let's do simple queries to print first rows of each table we will write a function to automate the process

In [4]:
# Creating a list of the tables:
tables = ['books', 'authors', 'ratings', 'reviews', 'publishers']

# Defining a function that will print first 5 tows from the table in the table list
def rows_5(tables):
    for table in tables:
        query = f''' SELECT *
                    FROM {table}
                    LIMIT 5
                '''
        df = pd.read_sql(query, con=engine)
        print(f"First 5 rows from table '{table}':")
        display(df)
        print('\n')
    return
# Applying the function
rows_5(tables)

First 5 rows from table 'books':


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




First 5 rows from table 'authors':


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




First 5 rows from table 'ratings':


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




First 5 rows from table 'reviews':


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...




First 5 rows from table 'publishers':


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


We see that all the tables are displayed correctly and we can start our tasks

## Task 1
We need to Find the number of books released after January 1, 2000.

In [5]:
# Creating a query
query1 = ''' SELECT COUNT(book_id)
                    FROM books
                    WHERE publication_date  > '2000-01-01'
                '''
# Executing the query and retrieving the count value
result1 = pd.read_sql(query1, con=engine)
count1 = result1.iloc[0, 0]  # Accessing the count value

# Printing the count value
print("Number of books published after 2000-01-01:", count1)

Number of books published after 2000-01-01: 819


## Task 2
We need to find the number of user reviews and the average rating for each book.

In [6]:
# Creating a query
query2 = ''' SELECT b.title AS title,
                    count(DISTINCT re.review_id) AS n_reviews,
                    ROUND(AVG(ra.rating), 1) AS avg_rating
            FROM books AS b
            LEFT JOIN reviews AS re ON re.book_id = b.book_id
            LEFT JOIN ratings AS ra ON ra.book_id = b.book_id
            GROUP BY b.book_id
            ORDER BY avg_rating DESC, n_reviews DESC
         '''
# Executing the query and retrieving the count value
result2 = pd.read_sql(query2, con=engine)

# Printing the count value
result2

,title,n_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.0
1,School's Out—Forever (Maximum Ride #2),3,5.0
2,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,Crucial Conversations: Tools for Talking When ...,2,5.0
4,Misty of Chincoteague (Misty #1),2,5.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.3
996,Drowning Ruth,3,2.0
997,His Excellency: George Washington,2,2.0
998,Junky,2,2.0


We got results for our 999 books. In addition we displayed the result in rating's descending order and then in number of reviews. We also rounded the average rating for better view.

## Task 3
We need to identify the publisher that has released the greatest number of books with more than 50 pages (this will help us exclude brochures and similar publications from the analysis).

In [7]:
# Creating a query
query3 = ''' SELECT p.publisher
             FROM books AS b
             RIGHT JOIN publishers AS p ON p.publisher_id = b.publisher_id
             WHERE b.num_pages > 50
             GROUP BY p.publisher
             ORDER BY COUNT(b.book_id) DESC
             LIMIT 1
         '''
# Executing the query and retrieving the name of the publisher
result3 = pd.read_sql(query3, con=engine)
name3 = result3.iloc[0,0]

# Printing the count value
print('The publisher that has released the greatest number of books with more than 50 pages:', name3)

The publisher that has released the greatest number of books with more than 50 pages: Penguin Books


## Task 4
We need to identify the author with the highest average book rating (look only at books with at least 50 ratings)

In [8]:
# Creating a query
query4 = ''' SELECT a.author, AVG(br.avg_rating)
            FROM authors AS a
            LEFT JOIN books AS b ON a.author_id = b.author_id
            INNER JOIN (
                      SELECT ra.book_id, AVG(ra.rating) AS avg_rating
                      FROM ratings AS ra
                      GROUP BY ra.book_id
                      HAVING COUNT(ra.rating_id) >=50
                      ) AS br ON br.book_id = b.book_id
            GROUP BY a.author
            ORDER BY AVG(br.avg_rating) DESC
            LIMIT 1
         '''
# Executing the query and retrieving the name of the author
result4 = pd.read_sql(query4, con=engine)
name4 = result4.iloc[0,0]

# Printing the count value
print('The author with the highest average book rating:', name4)

The author with the highest average book rating: J.K. Rowling/Mary GrandPré


## Task 5
We need to find the average number of text reviews among users who rated more than 50 books

In [9]:
# Creating a query
query5 = ''' SELECT ROUND(AVG(n_reviews)) 
             FROM (
                  SELECT DISTINCT u.username, COUNT(DISTINCT ra.book_id), COUNT(DISTINCT re.review_id) AS n_reviews
                  FROM ratings AS u
                  LEFT JOIN ratings AS ra ON ra.username = u.username
                  LEFT JOIN reviews AS re ON re.username = u.username
                  GROUP BY u.username
                  HAVING COUNT(DISTINCT ra.book_id) >50
                  ) AS subquery
         '''
# Executing the query and retrieving the average number of reviews
result5 = pd.read_sql(query5, con=engine)
number5 = result5.iloc[0,0]

# Printing the count value
print('The average number of text reviews among users who rated more than 50 books:', number5)

The average number of text reviews among users who rated more than 50 books: 24.0


# Overall conclusion
We were an analyst of the startup that develops new apps for book lovers. The purposes of the analysis were to make some queries from our data base that can help a value proposition for a new product. We have successfully established connection with the data base using pandas and sqlalchemy libraries.

We have found out that there are **819 books** that were published after 2000-01-01. <br>
We have calculated the number of user reviews and average rating for each book. The book with the highest rating and the maximum number of reviews is ` Dirty Job (Grim Reaper #1)` it has **4 reviews. and average rating 5.0** <br> We identified the publisher that has released the greatest number of books with more than 50 pages. She is `Penguin Books` who has released **42 books.**<br> We have found the author with the highest average book rating. They are `J.K. Rowling/Mary GrandPré`, their average book **rating is 4.3** <br>And finally we discovered that users who rated more than 50 books do **24 reviews** in average.